""" for further reading refer to the urls below 
simple one: https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-python
advanced : https://github.com/Azure-Samples/storage-blob-python-getting-started/blob/master/blob_advanced_samples.py
"""

In [1]:
from datetime import datetime, timedelta
from urllib.request import urlretrieve

import os, uuid, sys
from azure.storage.blob import BlockBlobService, PublicAccess

from __future__ import print_function
import datetime
import io
import time


import azure.storage.blob as azureblob
import azure.batch.batch_service_client as batch
import azure.batch.batch_auth as batchauth

from azure.storage.blob import (
    BlockBlobService,
    BlobPermissions
)
from azure.storage.blob import (
    ContentSettings,
    BlobBlock,
    BlockListType,
)

from datetime import datetime, timedelta

from azure.storage.blob import (
    BlockBlobService,
    ContainerPermissions,
)

In [2]:
"""
### to not share your keys visuable in the notebook, protect your keys by constructing your own json keys save document
### only use the below to construct your own keys document for the first time, uncomment to use
import json

data = {}  
data['keys'] = []  
data['keys'].append({  
    "_STORAGE_ACCOUNT_KEY": <your_azure_storage_key>',
    "_STORAGE_ACCOUNT_NAME": <the_storage_acc_name>,
    "containerName": <the_container_name>
})
print(data)
with open('storage.txt', 'w') as outfile:  
    json.dump(data, outfile)
    """

'\n### to not share your keys visuable in the notebook, protect your keys by constructing your own json keys save document\n### only use the below to construct your own keys document for the first time, uncomment to use\nimport json\n\ndata = {}  \ndata[\'keys\'] = []  \ndata[\'keys\'].append({  \n    "_STORAGE_ACCOUNT_KEY": <your_azure_storage_key>\',\n    "_STORAGE_ACCOUNT_NAME": <the_storage_acc_name>,\n    "containerName": <the_container_name>\n})\nprint(data)\nwith open(\'storage.txt\', \'w\') as outfile:  \n    json.dump(data, outfile)\n    '

In [3]:
import json
from pprint import pprint

with open('storage.txt') as f:
    data = json.load(f)


pprint(data['keys'][0]['containerName'])

'pythonin'


In [4]:
# Create the blob client, for use in obtaining references to
# blob storage containers and uploading files to containers.
_STORAGE_ACCOUNT_NAME=data['keys'][0]['_STORAGE_ACCOUNT_NAME']
_STORAGE_ACCOUNT_KEY=data['keys'][0]['_STORAGE_ACCOUNT_KEY']
containerName=data['keys'][0]['containerName'] # the container name you are interested in

### this is where you create the connection by supplying azureblob.BlockBlobService ( acc_name , acc_key)

blob_client = azureblob.BlockBlobService(
    account_name=_STORAGE_ACCOUNT_NAME,
    account_key=_STORAGE_ACCOUNT_KEY)
# check the connection is established by list the name of the container in the blob storage
print([blob.name for blob in blob_client.list_containers() ])

['pythonin', 'pythonout']


In [5]:
##### create containers 
input_container_name = 'pythonin'
output_container_name = 'pythonout'
blob_client.create_container(input_container_name, fail_on_exist=False)
blob_client.create_container(output_container_name, fail_on_exist=False)
print([blob.name for blob in blob_client.list_containers() ])

['pythonin', 'pythonout']


In [6]:
### get sas token and url
def get_container_sas_token(block_blob_client,
                            container_name, blob_permissions):
    """
    Obtains a shared access signature granting the specified permissions to the
    container.

    :param block_blob_client: A blob service client.
    :type block_blob_client: `azure.storage.blob.BlockBlobService`
    :param str container_name: The name of the Azure Blob storage container.
    :param BlobPermissions blob_permissions:
    :rtype: str
    :return: A SAS token granting the specified permissions to the container.
    """
    # Obtain the SAS token for the container, setting the expiry time and
    # permissions. In this case, no start time is specified, so the shared
    # access signature becomes valid immediately. Expiration is in 2 hours.
    container_sas_token = \
        block_blob_client.generate_container_shared_access_signature(
            container_name,
            permission=blob_permissions,
            expiry=datetime.utcnow() + timedelta(hours=2))
       

    return container_sas_token



In [7]:
def get_container_sas_url(block_blob_client,
                            container_name, blob_permissions):
    """
    Obtains a shared access signature URL that provides write access to the 
    ouput container to which the tasks will upload their output.

    :param block_blob_client: A blob service client.
    :type block_blob_client: `azure.storage.blob.BlockBlobService`
    :param str container_name: The name of the Azure Blob storage container.
    :param BlobPermissions blob_permissions:
    :rtype: str
    :return: A SAS URL granting the specified permissions to the container.
    """
    # Obtain the SAS token for the container.
    sas_token = get_container_sas_token(block_blob_client,
                            container_name, azureblob.BlobPermissions.WRITE)

    # Construct SAS URL for the container
    container_sas_url = "https://{}.blob.core.windows.net/{}?{}".format(_STORAGE_ACCOUNT_NAME, container_name, sas_token)

    return container_sas_url

In [8]:
### this is how to use get_container_sas_url function above by manipulating policy used 
policy=azureblob.BlobPermissions.READ+azureblob.BlobPermissions.READ
su=get_container_sas_url(blob_client,input_container_name, policy )
print(su[:10]+'*****')

https://bl*****


In [9]:
def properties_and_metadata_operations(create_container_name,account, local_file_path, deletYN):

        file_blob_name = "tinycat.png"

        text_blob_name = "Text"

         

        # Create a Block Blob Service object

        #blockblob_service = account.create_block_blob_service()



        container_name = create_container_name



        try:

            # Create a new container

            print('1. Create a container with name and custom metadata - ' + container_name)

            account.create_container(container_name, {'sample':'azure-storage'})

                    

            # Upload file as a block blob

            print('2. Uploading BlockBlob from file with properties and custom metadata')

            #Get full path on drive to file_to_upload by joining the fully qualified directory name and file name on the local drive

            full_path_to_file = os.path.join(local_file_path, file_blob_name)

            

            account.create_blob_from_path(container_name, file_blob_name, full_path_to_file, 

                content_settings=ContentSettings(content_type='application/png'),

                metadata={'category':'azure-samples'})

            

            account.create_blob_from_text(container_name, text_blob_name, 'Data',

                content_settings=ContentSettings(content_encoding ='UTF-8', content_language='en'),

                metadata={'origin':'usa', 'title': 'azure-samples'})

            

            # Get all the container properties 

            print('3. Get Container metadata')



            container = account.get_container_properties(container_name)

            

            print('    Metadata:')



            for key in container.metadata:

                print('        ' + key + ':' + container.metadata[key])

            

            # Get all the blob properties 

            print('4. Get Blob properties')

            blob = account.get_blob_properties(container_name, file_blob_name)

            

            print('    Metadata:')

            for key in blob.metadata:

                print('        ' + key + ':' + blob.metadata[key])

            

            print('    Properties:')

            print('        Content-Type:' + blob.properties.content_settings.content_type)

        finally:            

            # Delete the container

            print("5. Delete Container :", str(deletYN))

            if account.exists(container_name) and (deletYN):

                account.delete_container(container_name)
                print("container :{} deleted".format(container_name))

In [10]:
local_path=os.path.expanduser("~/Desktop/zeno_azure_toolkit/")
properties_and_metadata_operations('testpropmeta',blob_client,local_path, True)
print("double check it is deleted in azure")
print([blob.name for blob in blob_client.list_containers() ])

1. Create a container with name and custom metadata - testpropmeta
2. Uploading BlockBlob from file with properties and custom metadata
3. Get Container metadata
    Metadata:
        sample:azure-storage
4. Get Blob properties
    Metadata:
        category:azure-samples
    Properties:
        Content-Type:application/png
5. Delete Container : True
container :testpropmeta deleted
double check it is deleted in azure
['pythonin', 'pythonout']


In [11]:
### upload files from local PC up to blob 
def upload_file_to_container(block_blob_client, container_name, file_path):
    """
    Uploads a local file to an Azure Blob storage container.

    :param block_blob_client: A blob service client.
    :type block_blob_client: `azure.storage.blob.BlockBlobService`
    :param str container_name: The name of the Azure Blob storage container.
    :param str file_path: The local path to the file.
    :rtype: `azure.batch.models.ResourceFile`
    :return: A ResourceFile initialized with a SAS URL appropriate for Batch
    tasks.
    """
    blob_name = os.path.basename(file_path)

    print('Uploading file {} to container [{}]...'.format(file_path,
                                                          container_name))

    block_blob_client.create_blob_from_path(container_name,
                                            blob_name,
                                            file_path)
    
    # Obtain the SAS token for the container.
    sas_token = get_container_sas_token(block_blob_client,
                            container_name, azureblob.BlobPermissions.READ+azureblob.BlobPermissions.READ)
    # syntax ContainerPermissions(read=False, write=False, delete=False, list=False, _str=None)
    
    

    sas_url = block_blob_client.make_blob_url(container_name,
                                              blob_name,
                                              sas_token=sas_token)

    return blob_name, sas_token,sas_url

In [12]:
print(datetime.utcnow())
print(timedelta(2))

2018-05-09 11:30:56.194182
2 days, 0:00:00


In [13]:
#### let's actually upload some files to the blob in the pythonin container  created above 
local_path=os.path.expanduser("~/Desktop/batch-python-ffmpeg-tutorial/src/")
#print([fil for fil in os.listdir(local_path+'InputFiles')])


input_file_paths = []
        
for folder, subs, files in os.walk(local_path+'InputFiles/'):
    for filename in files:
        if filename.endswith(".mp4"):
            input_file_paths.append(os.path.abspath(os.path.join(folder, filename)))

            
# Upload the input files. This is the collection of files that are to be processed by the tasks. 
uploaded_file_tokens={}
for file_path in input_file_paths:
    
    #if file_path.endswith('1.mp4'):
        #print(file_path)
    bn,sast,sasu=upload_file_to_container(blob_client, input_container_name, file_path)
    #print("bn",bn)
    #print("sastoken", sast)
    #print("sas url", sasu)
    uploaded_file_tokens[bn]=[sast,sasu]
        
#input_files = [
#    upload_file_to_container(blob_client, input_container_name, file_path)
#    for file_path in input_file_paths]


### check the files are uploaded sucessfully 
print([blob.name for blob in blob_client.list_blobs(input_container_name)])

Uploading file C:\Users\zecharpy\Desktop\batch-python-ffmpeg-tutorial\src\InputFiles\LowPriVMs-1.mp4 to container [pythonin]...
Uploading file C:\Users\zecharpy\Desktop\batch-python-ffmpeg-tutorial\src\InputFiles\LowPriVMs-2.mp4 to container [pythonin]...
Uploading file C:\Users\zecharpy\Desktop\batch-python-ffmpeg-tutorial\src\InputFiles\LowPriVMs-3.mp4 to container [pythonin]...
Uploading file C:\Users\zecharpy\Desktop\batch-python-ffmpeg-tutorial\src\InputFiles\LowPriVMs-4.mp4 to container [pythonin]...
Uploading file C:\Users\zecharpy\Desktop\batch-python-ffmpeg-tutorial\src\InputFiles\LowPriVMs-5.mp4 to container [pythonin]...
['LowPriVMs-1.mp4', 'LowPriVMs-2.mp4', 'LowPriVMs-3.mp4', 'LowPriVMs-4.mp4', 'LowPriVMs-5.mp4']


In [14]:
list(uploaded_file_tokens.keys())[0], os.path.dirname('LowPriVMs-1.mp4'), os.listdir()

('LowPriVMs-1.mp4',
 '',
 ['.ipynb_checkpoints',
  '3.jpg',
  'azureblob4python.yml',
  'azure_ComputerVision_pythonAPI.ipynb',
  'handwritter_test.jpg',
  'resize.jpg',
  'storage.txt',
  'tinycat.PNG',
  'upload_files_to_blob_containers_via_SAS_token.ipynb',
  'video.jpg'])

In [15]:
# read this below
#https://github.com/Azure-Samples/storage-blob-python-getting-started/blob/master/blob_advanced_samples.py
#### this is how you download files=blobs from a container 

blob_client.get_blob_to_path(input_container_name, 'LowPriVMs-1.mp4', os.path.join(local_path,'LowPriVMs-1.mp4'  + '.copy.mp4'))
#####confirm the file with .copy as extension is indeed there 
for f in os.listdir(local_path):
    if f.endswith('.copy.mp4'):
        print("confirmed file :{} in directory :{} is found".format(f, local_path))
        ## comment out os.remove if you want to keep the files downloaded from  the blob
        os.remove(local_path+f)


confirmed file :LowPriVMs-1.mp4.copy.mp4 in directory :C:\Users\zecharpy/Desktop/batch-python-ffmpeg-tutorial/src/ is found


In [16]:
#### copying from one container to another

def copy_blob(blob_service,copy_from_container, copy_to_container, blobname_to_copy, deletOldYN):
    blob_service.create_container(copy_to_container, fail_on_exist=False)
    blob_url = blob_service.make_blob_url(copy_from_container,blobname_to_copy)
    # blob_url:https://demostorage.blob.core.windows.net/image-container/pretty.jpg

    blob_service.copy_blob(copy_to_container, blobname_to_copy, blob_url)

    if deletOldYN:
        #for move the file use this line
        blob_service.delete_blob(copy_from_container, blob_name)

In [17]:
copy_blob(blob_client, 'pythonin','copypythonin','LowPriVMs-1.mp4',False)

print([container.name for container in blob_client.list_containers()])
### clean up by deleting the extra container 
print("clean up by deleting the extra container we created for copying ", 'copypythonin')
print(blob_client.delete_container('copypythonin'))

['copypythonin', 'pythonin', 'pythonout']
clean up by deleting the extra container we created for copying  copypythonin
True
